In [13]:
import pandas as pd
import re
from datetime import datetime

In [14]:
df = pd.read_csv("../data/raw/Olympic_Event_Results.csv")
df.head()

,result_id,event_title,edition,edition_id,sport,sport_url,result_date,result_location,result_participants,result_format,result_detail,result_description
0,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"25 August 2004 — 16:30 (B), 20:00 (A)","Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,na,"Not so much a competition as a coronation, the..."
1,1626,"Giant Slalom, Women1",1998 Winter Olympics,46,Snowboarding,/editions/46/sports/SBD,9 February 1998,"Mt. Yakebitai, Shiga Kogen, Yamanouchi",31 from 14 countries,"Two runs, total time determined placement.",Gates: 38 / 36Length: 936 mStart Altitude: 196...,The women’s giant slalom was postponed one day...
2,76,"Singles, Men",1976 Winter Olympics,40,Luge,/editions/40/sports/LUG,4 – 7 February 1976,"Kunsteis-Bob- und Rodelbahn, Igls",43 from 15 countries,"Four runs, total time determined placement.",Curves: 14Length: 1220 mStart Altitude: ?Verti...,"Once more, the competitors from East and West ..."
3,962,"1,500 metres, Men",1928 Winter Olympics,30,Speed Skating,/editions/30/sports/SSK,14 February 1928 — 9:00,"Olympia-Eisstadion Badrutts Park, St. Moritz",30 from 14 countries,na,na,There was little doubt that the Olympic 1500 m...
4,258824,"Canadian Singles, Slalom, Men",2008 Summer Olympics,53,Canoe Slalom,/editions/53/sports/CSL,11 – 12 August 2008,"Shunyi Aolinpike Shuishang Gongyuan, Mapo, Shunyi",16 from 16 countries,na,na,Two former Olympic champions in the C-1 slalom...


In [15]:
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

def formatted_date(day, month, year):
    return f"{year}-{month_mapping[month]:02d}-{int(day):02d}"



def process_result_date(date_str):
    if(pd.isna(date_str)):
        return None, None
    
    # Define patterns
    single_date_pattern = r'\s*(\d{1,2})\s+(\w+)\s+(\d{4})'  # Matches '20 July 1904'
    range_within_same_month_pattern = r'\s*(\d{1,2})\s*–\s*(\d{1,2})\s+(\w+)\s+(\d{4})'  # Matches '1 – 11 August 2012'
    range_across_month_pattern = r'\s*(\d{1,2})\s+(\w+)\s*–\s*(\d{1,2})\s+(\w+)\s+(\d{4})'  # Matches '31 July – 1 August 1992'
    date_with_time_pattern = r'\s*(\d{1,2})\s+(\w+)\s+(\d{4})\s*-'  # Matches '11 August 2012 - 15:00'

    match = re.match(single_date_pattern, date_str)
    if match:
        day, month, year = match.groups()
        start_date = end_date = formatted_date(day, month, year)
        return start_date, end_date

    match = re.match(range_within_same_month_pattern, date_str)
    if match:
        start_day, end_day, month, year = match.groups()
        start_date = formatted_date(start_day, month, year)
        end_date = formatted_date(end_day, month, year)
        return start_date, end_date
    
    match = re.match(range_across_month_pattern, date_str)
    if match:
        start_day, start_month, end_day, end_month, year = match.groups()
        start_date = formatted_date(start_day, start_month, year)
        end_date = formatted_date(end_day, end_month, year)
        return start_date, end_date

   
    match = re.match(date_with_time_pattern, date_str)
    if match:
        day, month, year = match.groups()
        start_date = end_date = formatted_date(day, month, year)
        return start_date, end_date
    
    return None, None

df['start_date'], df['end_date'] = zip(*df['result_date'].apply(process_result_date))

print(df[['start_date', 'end_date', 'result_date']])

      start_date    end_date                            result_date
0     2004-08-25  2004-08-25  25 August 2004 — 16:30 (B), 20:00 (A)
1     1998-02-09  1998-02-09                        9 February 1998
2     1976-02-04  1976-02-07                   4 –  7 February 1976
3     1928-02-14  1928-02-14                14 February 1928 — 9:00
4     2008-08-11  2008-08-12                    11 – 12 August 2008
...          ...         ...                                    ...
7389  2021-07-25  2021-08-07               25 July –  7 August 2021
7390  1936-08-12  1936-08-13                    12 – 13 August 1936
7391  1900-05-14  1900-05-21                       14 – 21 May 1900
7392  2008-08-09  2008-08-21                     9 – 21 August 2008
7393  2000-09-16  2000-09-23                 16 – 23 September 2000

[7394 rows x 3 columns]


In [ ]:
df.drop(columns=['result_date'], inplace=True)
df.head()


,result_id,event_title,edition,edition_id,sport,sport_url,result_location,result_participants,result_format,result_detail,result_description,start_date,end_date
0,30359,"Super-Heavyweight (>105 kilograms), Men",2004 Summer Olympics,26,Weightlifting,/editions/26/sports/WLF,"Olympiako Gymnastirio Arsis Varon Nikaias, Nikaia",17 from 15 countries,Total of best lifts in snatch and clean & jerk...,na,"Not so much a competition as a coronation, the...",2004-08-25,2004-08-25
1,1626,"Giant Slalom, Women1",1998 Winter Olympics,46,Snowboarding,/editions/46/sports/SBD,"Mt. Yakebitai, Shiga Kogen, Yamanouchi",31 from 14 countries,"Two runs, total time determined placement.",Gates: 38 / 36Length: 936 mStart Altitude: 196...,The women’s giant slalom was postponed one day...,1998-02-09,1998-02-09
2,76,"Singles, Men",1976 Winter Olympics,40,Luge,/editions/40/sports/LUG,"Kunsteis-Bob- und Rodelbahn, Igls",43 from 15 countries,"Four runs, total time determined placement.",Curves: 14Length: 1220 mStart Altitude: ?Verti...,"Once more, the competitors from East and West ...",1976-02-04,1976-02-07
3,962,"1,500 metres, Men",1928 Winter Olympics,30,Speed Skating,/editions/30/sports/SSK,"Olympia-Eisstadion Badrutts Park, St. Moritz",30 from 14 countries,na,na,There was little doubt that the Olympic 1500 m...,1928-02-14,1928-02-14
4,258824,"Canadian Singles, Slalom, Men",2008 Summer Olympics,53,Canoe Slalom,/editions/53/sports/CSL,"Shunyi Aolinpike Shuishang Gongyuan, Mapo, Shunyi",16 from 16 countries,na,na,Two former Olympic champions in the C-1 slalom...,2008-08-11,2008-08-12


In [17]:
df.to_csv('../data/processed/Olympic_Event_Results.csv', index=False)

In [18]:
# from sqlalchemy import create_engine

# # Extract the values from Django settings
# DB_NAME = 'olympic'
# DB_USER = 'root'
# DB_PASSWORD = 'Maria%403306'
# DB_HOST = '127.0.0.1'
# DB_PORT = '3306'

# # Create the SQLAlchemy engine
# engine = create_engine(f'mariadb+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

# df = pd.read_csv('../data/processed/Olympic_Event_Results.csv')



In [19]:
# df.to_sql('Olympic_Event_Results', con=engine, index=False, if_exists='replace')